In [6]:
import sqlite3
import pandas as pd
from soynlp.noun import LRNounExtractor_v2
from collections import defaultdict
import re
import numpy as np
import math
from string import punctuation
from soynlp.word import WordExtractor
from soynlp.utils import DoublespaceLineCorpus

# Stopwords 처리
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation))) # punctuation 제거
pattern2 = re.compile(r'[^가-힣 ]') # 특수문자, 모음, 숫자, 영어 제거
pattern3 = re.compile(r'\s{2,}') # white space 1개로 바꾸기.

class Extracter:
    def __init__(self, df):
        self.df = df
        self.noun_extractor = LRNounExtractor_v2(verbose=True)
        self.word_extractor = WordExtractor(min_frequency=math.floor(len(self.df)*0.0001))
        
    def cleaning(self):
        self.df['head'] = self.df['head'].map(lambda x:pattern3.sub(' ',
                                                        pattern2.sub('',
                                                         pattern1.sub('', x))))
        return self.df

    def extract_nouns(self):
        tempary = np.linspace(0,1,11)
        nouns = [self.noun_extractor.train_extract(_['head'], min_noun_frequency=math.floor(len(self.df)*0.0001)) for _ in [self.df.iloc[math.ceil(len(self.df)*tempary[_]):math.ceil(len(self.df)*tempary[_+1])] for _ in range(len(tempary)-1)]]
        words = {k:v for i in range(len(nouns)) for k,v in nouns[i].items() if len(k) > 1}
        return words

    def search_dict(self,nouns):
        # 사전 검색 결과 없는 단어 추출
        conn = sqlite3.connect('kr_korean.db')
        cur = conn.cursor()
        data = pd.read_sql('SELECT word FROM kr', conn)
        data = ' '.join(data['word'])
        return pd.DataFrame([_ for _ in nouns if _[0] not in data])
    
    # 의미 추출을 위한 training data set 생성
    def extract_sent(self, words):
        sent = defaultdict(lambda:0)
        for w in new_words[0]:
            temp = [s for s in df['head'] if w in s]
            sent[w] = '  '.join(temp)
        return sent
            
    def extract_statistic_value(self, sent):
        statistic = defaultdict(lambda:0)
        for k,v in sent.items():
            self.word_extractor.train([v])
            try:
                statistic[k] = self.word_extractor.extract()[k]
            except Exception as e:
                print(e)
        return statistic
    
    def extract_r_rat(self, sent, statistic):
        conn = sqlite3.connect('kr_korean.db')
        cur = conn.cursor()
        post_pos = pd.read_sql('SELECT word FROM kr WHERE part="조사"', conn)
        post_pos['word'] = post_pos['word'].map(lambda x: pattern3.sub(' ',
                                                            pattern2.sub('',
                                                             pattern1.sub('', x))))
        post_pos.drop_duplicates(keep='first', inplace=True)
        post_pos = ''.join(post_pos['word'])
        r_rat = defaultdict(lambda:0)
        for k in statistic.keys():
            try:
                self.noun_extractor.train_extract([sent[k]])
                count = pprat = wsrat = 0
                for _ in self.noun_extractor.lrgraph.get_r(k, topk=-1):
                    if _[0] in post_pos:
                        if _[0] != '':
                            pprat += _[1]
                        elif _[0] == '':
                            wsrat = _[1]
                for _ in self.noun_extractor.lrgraph.get_r(k, topk=-1):
                    count += _[1]

                r_rat[k] = {'rpprat':pprat/count, 'rwsrat':wsrat/count}
            except Exception as e:
                print(e)
        return r_rat

In [7]:
conn = sqlite3.connect('Humor.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head',conn)
df.drop_duplicates(keep='first', inplace=True)
ext = Extracter(df)
df = ext.cleaning()

new_words = ext.search_dict(sorted(ext.extract_nouns().items(),key=lambda _:_[1], reverse=True))
sent = ext.extract_sent(new_words)

# 변수 생성
statistic = ext.extract_statistic_value(sent)
# rpprat : 명사의 오른쪽에 조사가 오는 비율, rwsrat : 명사의 오른쪽에 white space가 오는 비율
r_rat = ext.extract_r_rat(sent,statistic)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 153629 from 127907 sents. mem=0.836 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=556166, mem=0.842 Gb
[Noun Extractor] batch prediction was completed for 40185 words
[Noun Extractor] checked compounds. discovered 27173 compounds
[Noun Extractor] postprocessing detaching_features : 426 -> 423
[Noun Extractor] postprocessing ignore_features : 423 -> 397
[Noun Extractor] postprocessing ignore_NJ : 397 -> 395
[Noun Extractor] 395 nouns (27173 compounds) with min frequency=127
[Noun Extractor] flushing was done. mem=0.879 Gb                    
[Noun Extractor] 31.14 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 156261 from 127907 sents. mem=0.933 Gb                    
[Noun Extractor] complete eojeol counter -> lr grap

all branching entropies was computed # words = 21929
all accessor variety was computed # words = 21929
training was done. used memory 1.118 Gb1.118 Gb
all cohesion probabilities was computed. # words = 170
all branching entropies was computed # words = 25059
all accessor variety was computed # words = 25059
training was done. used memory 1.118 Gb1.118 Gb
all cohesion probabilities was computed. # words = 179
all branching entropies was computed # words = 26184
all accessor variety was computed # words = 26184
training was done. used memory 1.118 Gb1.118 Gb
all cohesion probabilities was computed. # words = 205
all branching entropies was computed # words = 28439
all accessor variety was computed # words = 28439
training was done. used memory 1.118 Gb1.118 Gb
all cohesion probabilities was computed. # words = 205
all branching entropies was computed # words = 28968
all accessor variety was computed # words = 28968
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilitie

all branching entropies was computed # words = 62052
all accessor variety was computed # words = 62052
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 744
all branching entropies was computed # words = 62144
all accessor variety was computed # words = 62144
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 749
all branching entropies was computed # words = 62195
all accessor variety was computed # words = 62195
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 756
all branching entropies was computed # words = 62272
all accessor variety was computed # words = 62272
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 759
all branching entropies was computed # words = 62331
all accessor variety was computed # words = 62331
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilitie

all branching entropies was computed # words = 70253
all accessor variety was computed # words = 70253
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 919
all branching entropies was computed # words = 70391
all accessor variety was computed # words = 70391
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 922
all branching entropies was computed # words = 70421
all accessor variety was computed # words = 70421
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 925
all branching entropies was computed # words = 70456
all accessor variety was computed # words = 70456
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 928
all branching entropies was computed # words = 70524
all accessor variety was computed # words = 70524
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilitie

training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 1089
all branching entropies was computed # words = 76805
all accessor variety was computed # words = 76805
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 1092
all branching entropies was computed # words = 76860
all accessor variety was computed # words = 76860
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 1094
all branching entropies was computed # words = 76881
all accessor variety was computed # words = 76881
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 1097
all branching entropies was computed # words = 77212
all accessor variety was computed # words = 77212
training was done. used memory 1.128 Gb1.128 Gb
all cohesion probabilities was computed. # words = 1106
all branching entropies was computed # words = 77262
all accessor va

...

[Noun Extractor] flushing was done. mem=1.093 Gb                    
[Noun Extractor] 40.73 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 3106 from 1 sents. mem=1.093 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=5327, mem=1.093 Gb
[Noun Extractor] batch prediction was completed for 1012 words
[Noun Extractor] checked compounds. discovered 24 compounds
[Noun Extractor] postprocessing detaching_features : 267 -> 267
[Noun Extractor] postprocessing ignore_features : 267 -> 255
[Noun Extractor] postprocessing ignore_NJ : 255 -> 255
[Noun Extractor] 255 nouns (24 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.093 Gb                    
[Noun Extractor] 39.93 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 682 from 1 sents. mem=1.093 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extrac

[Noun Extractor] checked compounds. discovered 97 compounds
[Noun Extractor] postprocessing detaching_features : 399 -> 377
[Noun Extractor] postprocessing ignore_features : 377 -> 355
[Noun Extractor] postprocessing ignore_NJ : 355 -> 354
[Noun Extractor] 354 nouns (97 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.093 Gb                    
[Noun Extractor] 48.89 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1850 from 1 sents. mem=1.093 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=3215, mem=1.093 Gb
[Noun Extractor] batch prediction was completed for 698 words
[Noun Extractor] checked compounds. discovered 8 compounds
[Noun Extractor] postprocessing detaching_features : 149 -> 149
[Noun Extractor] postprocessing ignore_features : 149 -> 140
[Noun Extractor] postprocessing ignore_NJ : 140 -> 140
[Noun Extractor] 140 nouns (8 compounds) with min fre

[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=7728, mem=1.093 Gb
[Noun Extractor] batch prediction was completed for 1245 words
[Noun Extractor] checked compounds. discovered 86 compounds
[Noun Extractor] postprocessing detaching_features : 470 -> 453
[Noun Extractor] postprocessing ignore_features : 453 -> 435
[Noun Extractor] postprocessing ignore_NJ : 435 -> 434
[Noun Extractor] 434 nouns (86 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.093 Gb                    
[Noun Extractor] 46.49 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 2399 from 1 sents. mem=1.093 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=3832, mem=1.093 Gb
[Noun Extractor] batch prediction was completed for 778 words
[Noun Extractor] checked compounds. discovered 42 compounds
[Noun Extractor] postprocessing detaching_features : 

[Noun Extractor] flushing was done. mem=1.093 Gb                    
[Noun Extractor] 43.90 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 5164 from 1 sents. mem=1.093 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=8252, mem=1.093 Gb
[Noun Extractor] batch prediction was completed for 1807 words
[Noun Extractor] checked compounds. discovered 35 compounds
[Noun Extractor] postprocessing detaching_features : 474 -> 473
[Noun Extractor] postprocessing ignore_features : 473 -> 458
[Noun Extractor] postprocessing ignore_NJ : 458 -> 458
[Noun Extractor] 458 nouns (35 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.093 Gb                    
[Noun Extractor] 24.44 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 2522 from 1 sents. mem=1.093 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extra

[Noun Extractor] postprocessing ignore_features : 125 -> 116
[Noun Extractor] postprocessing ignore_NJ : 116 -> 116
[Noun Extractor] 116 nouns (17 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.093 Gb                    
[Noun Extractor] 42.79 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1133 from 1 sents. mem=1.093 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=1965, mem=1.093 Gb
[Noun Extractor] batch prediction was completed for 357 words
[Noun Extractor] checked compounds. discovered 15 compounds
[Noun Extractor] postprocessing detaching_features : 98 -> 98
[Noun Extractor] postprocessing ignore_features : 98 -> 96
[Noun Extractor] postprocessing ignore_NJ : 96 -> 96
[Noun Extractor] 96 nouns (15 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.093 Gb                    
[Noun Extractor] 44.94 % eojeols are covered


In [8]:
r_rat

defaultdict(<function __main__.Extracter.extract_r_rat.<locals>.<lambda>()>,
            {'신천지': {'rpprat': 0.22842105263157894,
              'rwsrat': 0.6547368421052632},
             '확진자': {'rpprat': 0.17532325224632916,
              'rwsrat': 0.7565198334429104},
             '알아보자': {'rpprat': 0.007107133456172677,
              'rwsrat': 0.9648153022725279},
             '문재앙': {'rpprat': 0.20966010733452595,
              'rwsrat': 0.7220035778175313},
             '요즘': {'rpprat': 0.05871949062610541,
              'rwsrat': 0.8906968517863459},
             '재업': {'rpprat': 0.009609224855861628,
              'rwsrat': 0.5675848814862268},
             '너네': {'rpprat': 0.1800162910670649,
              'rwsrat': 0.7303828400760249},
             '아이돌': {'rpprat': 0.15474947807933195,
              'rwsrat': 0.7210334029227558},
             '얘들아': {'rpprat': 0.004133654839820875,
              'rwsrat': 0.9724423010678608},
             '스압': {'rpprat': 0.00490402129746392,